In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
%matplotlib inline

# %matplotlib inline # ligne pour jupyter seulement 

import json
import networkx as nx
from nltk.tokenize import TweetTokenizer


df = pd.read_csv('PART2_Infos_generales_Commentaire.csv', sep = ',')

print(len(df))

df.head()


3093


,Unnamed: 0,Titre comment,Date de réponse,Reponse ou non,Liste peronnes,Nombre d'étoile
0,0,2 mois de delai avant livraison et le…,05 mars 2023,True,Showroomprive.com,1
1,1,Je n'ai pas reçu le bon d'achat de…,05 mars 2023,True,Showroomprive.com,1
2,2,Je n est pas encore récupéré ma commande elle ...,04 mars 2023,True,Showroomprive.com,1
3,3,Ma commande est confirmé et tout…,04 mars 2023,True,Showroomprive.com,2
4,4,Réponse du service client tard et mauvaise,04 mars 2023,True,Showroomprive.com,1


In [90]:
# Suppression de Unamed

df.drop('Unnamed: 0', axis = 1)

,Titre comment,Date de réponse,Reponse ou non,Liste peronnes,Nombre d'étoile
0,2 mois de delai avant livraison et le…,05 mars 2023,True,Showroomprive.com,1
1,Je n'ai pas reçu le bon d'achat de…,05 mars 2023,True,Showroomprive.com,1
2,Je n est pas encore récupéré ma commande elle ...,04 mars 2023,True,Showroomprive.com,1
3,Ma commande est confirmé et tout…,04 mars 2023,True,Showroomprive.com,2
4,Réponse du service client tard et mauvaise,04 mars 2023,True,Showroomprive.com,1
...,...,...,...,...,...
3088,"Cher, décevant et malhonnête, à fuir.",02 janvier 2021,True,Showroomprive.com,2
3089,Lamentable..,02 janvier 2021,True,Showroomprive.com,1
3090,Site à ne pas recommander,01 janvier 2021,True,Showroomprive.com,1
3091,Achat de 8 chaises le 09/12/2020,09 décembre 2020,True,Showroomprive.com,1


In [91]:
missing_values = df.isna().sum()
print(missing_values)

Unnamed: 0         0
Titre comment      0
Date de réponse    0
Reponse ou non     0
Liste peronnes     4
Nombre d'étoile    0
dtype: int64


In [92]:
df.dropna()
missing_values2 = df.isna().sum()
print(missing_values2)

Unnamed: 0         0
Titre comment      0
Date de réponse    0
Reponse ou non     0
Liste peronnes     4
Nombre d'étoile    0
dtype: int64


In [93]:
commentaires = df['Titre comment']


from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer

sentiment_intensity_analyzer = SentimentIntensityAnalyzer()

score = commentaires.apply(lambda x: sentiment_intensity_analyzer.polarity_scores(x))
scores = commentaires.apply(lambda x: sentiment_intensity_analyzer.polarity_scores(x)['compound'])

print(scores.shape)
print(type(scores))
# print(scores)
scores.describe()

df_score = scores.to_frame()

df_score

(3093,)
<class 'pandas.core.series.Series'>


,Titre comment
0,0.0000
1,-0.2924
2,-0.6599
3,0.0000
4,-0.5574
...,...
3088,-0.5994
3089,-0.4039
3090,-0.2755
3091,0.0000


In [94]:
print("0")

0


In [95]:
#Définition des variables expplicatives et cibles

feats_int = df.drop(['Titre comment','Unnamed: 0', 'Date de réponse' ,"Nombre d'étoile"], axis=1)
feats_int.shape

feats_int.head()

,Reponse ou non,Liste peronnes
0,True,Showroomprive.com
1,True,Showroomprive.com
2,True,Showroomprive.com
3,True,Showroomprive.com
4,True,Showroomprive.com


In [96]:
array_feats = np.concatenate((feats_int, df_score), axis = 1)


print(array_feats) #.head()

[[True 'Showroomprive.com' 0.0]
 [True 'Showroomprive.com' -0.2924]
 [True 'Showroomprive.com' -0.6599]
 ...
 [True 'Showroomprive.com' -0.2755]
 [True 'Showroomprive.com' 0.0]
 [True 'Showroomprive.com' 0.0]]


In [97]:
# Convertir feats en DataFrame
feats = pd.DataFrame(array_feats, columns=list(feats_int.columns) + ["score"])

# Afficher le DataFrame

#feats.drop("Unnamed: 0", axis=1)
feats.head()

,Reponse ou non,Liste peronnes,score
0,True,Showroomprive.com,0.0
1,True,Showroomprive.com,-0.2924
2,True,Showroomprive.com,-0.6599
3,True,Showroomprive.com,0.0
4,True,Showroomprive.com,-0.5574


pandas.core.frame.DataFrame

In [99]:
print(0)

0


In [100]:
target = df["Nombre d'étoile"]
target.head()

0    1
1    1
2    1
3    2
4    1
Name: Nombre d'étoile, dtype: int64

In [101]:
#Séparation des variables en test et en entrainement

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.25, random_state = 42)




In [102]:
# Insérez votre code ici 
from sklearn.preprocessing import OneHotEncoder

# Le paramètre drop permet d'éviter le problème de multicolinéarité
ohe = OneHotEncoder( drop="first", sparse=False)

X_train.loc[:,['Reponse ou non', 'Liste peronnes']] = ohe.fit_transform(X_train[['Reponse ou non', 'Liste peronnes']])

X_test.loc[:,['Reponse ou non', 'Liste peronnes']] = ohe.transform(X_test[['Reponse ou non', 'Liste peronnes']])

X_test.head()

,Reponse ou non,Liste peronnes,score
2441,1.0,0.0,-0.4003
2218,1.0,0.0,-0.5743
2245,1.0,0.0,-0.3612
2624,1.0,0.0,-0.2484
2637,1.0,0.0,0.0


In [105]:
# Insérez votre code ici
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

score_train = clf.score(X_train, y_train)
score_test = clf.score(X_test, y_test)

print("score d'entrainement :", score_train, "////", "score de test :", score_test)


score d'entrainement : 0.8180250107805088 //// score de test : 0.7790697674418605


In [108]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)

print(pd.crosstab(y_test,y_pred))
print(classification_report(y_test, y_pred))

col_0              1   2
Nombre d'étoile         
1                598  14
2                157   5
              precision    recall  f1-score   support

           1       0.79      0.98      0.87       612
           2       0.26      0.03      0.06       162

    accuracy                           0.78       774
   macro avg       0.53      0.50      0.47       774
weighted avg       0.68      0.78      0.70       774



In [6]:
tokenizer = TweetTokenizer()
texte = ' '.join(commentaires)
mots = tokenizer.tokenize(texte)
print(len(mots))

from collections import Counter

word_count = Counter([m.lower() for m in mots])

mots_tries = sorted(word_count, key=lambda x: word_count.get(x), reverse=True)

17894


In [109]:
print(mots_tries)

['…', 'de', '!', 'commande', 'pas', 'je', 'la', 'et', 'un', 'une', '’', 'le', 'très', 'à', 'des', 'ai', 'a', 'ne', 'livraison', 'reçu', 'colis', "j'ai", 'j', 'en', 'service', 'déçue', 'sur', 'les', 'site', 'est', 'non', 'commandé', 'ma', 'du', '.', 'article', 'pour', 'client', ',', 'produit', 'n', 'que', 'déçu', '2', 'fuir', 'ce', 'mon', 'mauvaise', 'plus', 'suis', 'articles', 'qualité', 'jamais', 'showroom', 'retour', 'trop', 'bonjour', 'd', 'dans', '...', 'arnaque', 'produits', 'par', 'au', 'qui', 'commandes', 'avec', 'vente', 'il', 'été', 'mais', 'frais', 'achat', 'tout', 'acheté', 'expérience', 'mauvais', 'fois', 'conforme', 'nul', 'taille', 'l', 'showroomprive', 'sont', 'sav', 'chez', 'depuis', 'erreur', 'deux', 'après', 'attention', "n'ai", '1', 'déception', '3', 'car', 'arrivé', 'toujours', 'cliente', 'mois', 'montre', 'déplorable', 'commander', 'problème', 'dernière', 'sans', 'remboursement', 'fuyez', 'bon', "n'est", 'catastrophique', 'décevant', '€', 'prix', 'c', 'suite', 'mes